In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
(train_images, train_labels), (test_images, test_labels) = keras.datasets.fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
def plot_image(image):
  plt.figure(figsize = (2, 2))
  plt.imshow(image, cmap = "binary")
  plt.axis("off")

In [ ]:
def plot_multiple_images(images, num_img = 10, rows = 2, cols = 5):
  fig = plt.figure(figsize = (8, 4))
  for ind in range(1, num_img + 1):
    fig.add_subplot(rows, cols, ind)
    plt.imshow(images[ind - 1], cmap = "binary")
    plt.axis("off")

In [ ]:
X_train = train_images / 255.0
X_train = X_train.reshape((-1, 28, 28, 1))

In [ ]:
class_names = ["T-shirt/top", "Trousers", "Pullover", "Dress", "Coat", "Sandal", "Shirt", ""]

In [ ]:
latent_size = 50

generator = keras.models.Sequential([
  keras.layers.Dense(256 * 7 * 7, input_shape = [latent_size]),
  keras.layers.LeakyReLU(),
  keras.layers.Reshape([7, 7, 256]),

  keras.layers.Conv2DTranspose(128, 3, strides = 1, padding = "same"),
  keras.layers.LeakyReLU(),

  keras.layers.Conv2DTranspose(64, 3, strides = 2, padding = "same"),
  keras.layers.LeakyReLU(),

  keras.layers.Conv2DTranspose(32, 3, strides = 2, padding = "same"),
  keras.layers.LeakyReLU(),

  keras.layers.Conv2D(1, 3, strides = 1, padding = "same", activation = "sigmoid"),
])

discriminator = keras.models.Sequential([
  keras.layers.Conv2D(32, 3, strides = 2, padding = "same", input_shape = [28, 28, 1]),
  keras.layers.LeakyReLU(),
  keras.layers.Dropout(0.5),

  keras.layers.Conv2D(64, 3, padding = "same"),
  keras.layers.LeakyReLU(),
  keras.layers.Conv2D(64, 3, strides = 2, padding = "same"),
  keras.layers.LeakyReLU(),
  keras.layers.Dropout(0.5),

  keras.layers.Conv2D(128, 3, padding = "same"),
  keras.layers.LeakyReLU(),
  keras.layers.Dropout(0.5),

  keras.layers.GlobalMaxPooling2D(),
  keras.layers.Dense(1, activation = "sigmoid"),
])

gan = keras.models.Sequential([generator, discriminator])

keras.layers.Flatten(),
  keras.layers.Dense(128),
  keras.layers.LeakyReLU(),

  keras.layers.Dense(64),
  keras.layers.LeakyReLU(),
  keras.layers.Dropout(0.5),

In [ ]:
gan.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_3 (Sequential)    (None, 28, 28, 1)         1027329   
_________________________________________________________________
sequential_4 (Sequential)    (None, 1)                 129729    
Total params: 1,157,058
Trainable params: 1,157,058
Non-trainable params: 0
_________________________________________________________________


In [ ]:
optimizer = keras.optimizers.Adam(learning_rate = 0.0001)
discriminator.compile(optimizer = optimizer, loss = "binary_crossentropy")
gan.compile(optimizer = optimizer, loss = "binary_crossentropy")

In [ ]:
batch_size = 256

def train_gan(num_epochs):
  generator, discriminator = gan.layers
  for epoch in range(num_epochs):
    print(f"Epoch: {epoch + 1} / {num_epochs}")
    for i in range(0, X_train.shape[0] - batch_size, batch_size):
      #train discriminator
      noise = tf.random.normal(shape = [batch_size, latent_size])
      fake_images = generator(noise)
      real_images = X_train[i : i + batch_size]
      conc_images = np.concatenate((real_images, fake_images), axis = 0)
      conc_labels = np.concatenate((np.ones((batch_size, 1)), np.zeros((batch_size, 1))), axis = 0)
      discriminator.trainable = True
      discriminator.train_on_batch(conc_images, conc_labels)

      #train generator
      noise = tf.random.normal(shape = [batch_size, latent_size])
      labels = np.ones((batch_size, 1))
      discriminator.trainable = False
      gan.train_on_batch(noise, labels)

In [ ]:
train_gan(50)

Epoch: 1 / 50
Epoch: 2 / 50
Epoch: 3 / 50
Epoch: 4 / 50
Epoch: 5 / 50
Epoch: 6 / 50
Epoch: 7 / 50
Epoch: 8 / 50
Epoch: 9 / 50
Epoch: 10 / 50
Epoch: 11 / 50
Epoch: 12 / 50
Epoch: 13 / 50
Epoch: 14 / 50
Epoch: 15 / 50
Epoch: 16 / 50
Epoch: 17 / 50
Epoch: 18 / 50
Epoch: 19 / 50
Epoch: 20 / 50
Epoch: 21 / 50
Epoch: 22 / 50
Epoch: 23 / 50
Epoch: 24 / 50
Epoch: 25 / 50
Epoch: 26 / 50
Epoch: 27 / 50
Epoch: 28 / 50
Epoch: 29 / 50
Epoch: 30 / 50
Epoch: 31 / 50
Epoch: 32 / 50
Epoch: 33 / 50
Epoch: 34 / 50
Epoch: 35 / 50
Epoch: 36 / 50
Epoch: 37 / 50
Epoch: 38 / 50
Epoch: 39 / 50
Epoch: 40 / 50
Epoch: 41 / 50
Epoch: 42 / 50
Epoch: 43 / 50
Epoch: 44 / 50
Epoch: 45 / 50
Epoch: 46 / 50
Epoch: 47 / 50
Epoch: 48 / 50
Epoch: 49 / 50
Epoch: 50 / 50


In [ ]:
noise = tf.random.normal([1, latent_size])
image = generator(noise)
plot_image(np.squeeze(image))